In [1]:
import sys

# uncomment any library you need to install and run the cell

#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install sklearn
#!{sys.executable} -m pip install emoji
#!{sys.executable} -m pip install spacy
#!{sys.executable} -m pip install gensim

In [9]:
import os, re, gzip, json, pickle, shutil, random, joblib, csv, random

import spacy

import numpy as np
import matplotlib.pyplot as plt

from emoji import UNICODE_EMOJI

%matplotlib inline

In [3]:
#!{sys.executable} -m spacy download en_core_web_sm

In [ ]:
#
# Test module based off dev work
#

In [4]:
sys.path.append('../app')
from config import Config
from preprocessing import Preprocessing

In [5]:
config = Config()

In [6]:
preprocessor = Preprocessing(config)

In [7]:
preprocessor.extract_type_and_text()

Extracting raw troll tweets...
Extracted 2116866 troll tweets (badtabs: 0, badlines: 1, skipped nonenglish: 829340)
Extracting raw user tweets...
0
1000000
2000000
3000000
4000000
5000000
0
1000000
2000000
3000000
bad tabs: 217985, bad lines: 223
Extracted 8783464 normal user tweets (badtabs: 217985, badlines: 223)


In [7]:
preprocessor.clean_tweets()

Saving 2116866 cleaned troll tweets to ../data/troll_tweets_clean.pkl.gz
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
Saving 8783464 cleaned user tweets to ../data/user_tweets_clean.pkl.gz


In [8]:
with gzip.open(config.troll_tweet_clean_path, 'rb') as fz:
    troll_tweets = pickle.load(fz)
    
with gzip.open(config.user_tweet_clean_path, 'rb') as fz:
    user_tweets = pickle.load(fz)

In [10]:
#truncate and store smaller set
random.shuffle(troll_tweets)
config.troll_tweet_clean_path = '../data/troll_tweets_clean_small.pkl.gz'
with gzip.open(config.troll_tweet_clean_path, 'wb') as oz:
    pickle.dump(troll_tweets[:100000], oz)
        
random.shuffle(user_tweets)
config.user_tweet_clean_path = '../data/user_tweets_clean_small.pkl.gz'
with gzip.open(config.user_tweet_clean_path, 'wb') as oz:
    pickle.dump(user_tweets[:100000], oz)

In [11]:
preprocessor = Preprocessing(config)

In [12]:
preprocessor.tokenize_tweets()

Storing troll tweets tagged so far: 0 commercial	<LINK> So, did you crush your workout? <USER> @_kaskp <USER> <USER> <USER> <USER> <USER>	<LINK> So , did you crush your workout ? <USER> @_kaskp <USER> <USER> <USER> <USER> <USER>	<LINK> so , do you crush your workout ? <USER> @_kaskp <USER> <USER> <USER> <USER> <USER>	TAG RB , VBD PRP VB PRP$ NN . TAG NNP TAG TAG TAG TAG TAG	your_workout	
Storing complete 100000 tagged troll tweets
Storing user tweets tagged so far: 0 NormalUser	Congrats to Toby Warrior Shaye G. of New York, NY, the winner of the Toby Keith Norway Flyaway Sweepstakes! - <LINK>	Congrats to Toby Warrior Shaye G. of New York , NY , the winner of the Toby Keith Norway Flyaway Sweepstakes ! - <LINK>	congrat to Toby Warrior Shaye G. of New York , NY , the winner of the Toby Keith Norway Flyaway sweepstake ! - <LINK>	NNS IN NNP NNP NNP NNP IN NNP NNP , NNP , DT NN IN DT NNP NNP NNP NNP NNS . : TAG	Toby_Warrior_Shaye_G. New_York the_winner the_Toby_Keith_Norway_Flyaway_Sweepsta

In [ ]:
#
# Dev work
#

In [5]:
data_path = 'data'
troll_tweet_path = '%s/russian-troll-tweets' % data_path
norm_tweet_path = '%s/twitter_cikm_2010' % data_path

#extracted text
troll_tweet_texts_path = '%s/troll_tweets.pkl.gz' % data_path
user_tweet_texts_path = '%s/user_tweets.pkl.gz' % data_path

spacy_nlp = spacy.load('en_core_web_sm')

In [35]:
def get_tweets(troll_tweet_path, norm_tweet_path):
    troll_tweets = get_troll_tweets(troll_tweet_path)
    
    #training_set_tweets.txt, training_set_users.txt, test_set_tweets.txt, test_set_users.txt
    #15861944	5766883977	@techiepalar Please prove to me that you are a real human by following me based on this tweet? I quit TrueTwitValidation & won't use it.	2009-11-16 09:04:01
    norm_tweets = get_norm_tweets('%s/training_set_tweets.txt' % norm_tweet_path)
    print('%d norm tweets from training file' % len(norm_tweets))
    norm_tweets.extend(get_norm_tweets('%s/test_set_tweets.txt' % norm_tweet_path))
    print('%d total norm tweets' % len(norm_tweets))
    
    return troll_tweets, norm_tweets


def get_troll_tweets(troll_tweet_path):
    troll_tweets = []
    badtab=0
    badline=0
    nonenglish=0
    for fn in os.listdir(troll_tweet_path):
        if not fn.endswith('.csv'):
            continue
        with open('%s/%s' % (troll_tweet_path, fn), 'r', encoding='utf-8', newline='\n') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='"')
            header = next(reader)
            #external_author_id, author, content, region, language, publish_date, arvested_date, following,
            #followers, updates, post_type, account_type, retweet, account_category, new_june_2018, alt_external_id,
            # tweet_id, article_url, tco1_step1, tco2_step1, tco3_step1
            for fields in reader:
                if not len(fields)==21:
                    badtab+=1
                    continue
                    
                if not fields[4].lower().strip()=='english':
                    nonenglish+=1
                    continue
                
                text = fields[2].strip()
                if not text:
                    badline+=1
                    continue
                    
                troll_type = fields[13].strip()
                troll_tweets.append('%s\t%s' % (troll_type, text))
                
        print(fn, len(troll_tweets), badtab, badline, nonenglish)
    return troll_tweets

            
def get_norm_tweets(path):
    norm_tweets = []
    badtab=0
    badline=0
    
    with open(path, 'r', encoding='utf-8') as f:
        for i,line in enumerate(f):
            if i%250000==0:
                print(i)
            if not line.count('\t')==3:
                badtab+=1
                continue
            line = line.replace('\n','').split('\t')[2].strip()
            if not line:
                badline+=1
                continue
            norm_tweets.append('NormalUser\t%s' % line)
            
    print('bad tabs: %d, bad lines: %d' % (badtab, badline))
    
    return norm_tweets

In [36]:
if not os.path.exists(troll_tweet_texts_path):
    troll_tweets, norm_tweets = get_tweets(troll_tweet_path, norm_tweet_path)
    
    with gzip.open('%s/troll_tweets.pkl.gz' % data_path, 'wb') as oz:
        pickle.dump(troll_tweets, oz)
        
    with gzip.open('%s/normal_tweets.pkl.gz' % data_path, 'wb') as oz:
        pickle.dump(norm_tweets, oz)
else:
    with gzip.open('%s/troll_tweets.pkl.gz' % data_path, 'rb') as fz:
        troll_tweets = pickle.load(fz)
        
    with gzip.open('%s/normal_tweets.pkl.gz' % data_path, 'rb') as fz:
        norm_tweets = pickle.load(fz)
        
len(troll_tweets), len(norm_tweets)

IRAhandle_tweets_1.csv 243891 0 0 53639
IRAhandle_tweets_10.csv 501002 0 0 136284
IRAhandle_tweets_11.csv 751623 0 0 192906
IRAhandle_tweets_12.csv 990973 0 0 256290
IRAhandle_tweets_13.csv 1011867 0 0 265581
IRAhandle_tweets_2.csv 1262387 0 0 319645
IRAhandle_tweets_3.csv 1505289 0 0 348110
IRAhandle_tweets_4.csv 1752083 0 0 403696
IRAhandle_tweets_5.csv 1969306 0 0 460357
IRAhandle_tweets_6.csv 2227356 0 1 507917
IRAhandle_tweets_7.csv 2469708 0 1 602085
IRAhandle_tweets_8.csv 2712666 0 1 721318
IRAhandle_tweets_9.csv 2946206 0 1 829340
0
250000
500000
750000
1000000
1250000
1500000
1750000
2000000
2250000
2500000
2750000
3000000
3250000
3500000
3750000
bad tabs: 170147, bad lines: 176
3675302 norm tweets from training file
0
250000
500000
750000
1000000
1250000
1500000
1750000
2000000
2250000
2500000
2750000
3000000
3250000
3500000
3750000
4000000
4250000
4500000
4750000
5000000
bad tabs: 47838, bad lines: 47
8783464 total norm tweets


(2946206, 8783464)

In [41]:
# preproc cleaning 
# - replace links with <LINK>, replace usertags with <USER>
# - normalize spaces, quotes, double quotes, etc
def clean_tweets(troll_tweets, norm_tweets):
    clean_troll_tweets = []
    for i,troll_tweet in enumerate(troll_tweets):
        clean_troll_tweets.append(clean_tweet(troll_tweet))
        
    clean_norm_tweets = []
    for i,norm_tweet in enumerate(norm_tweets):
        clean_norm_tweets.append(clean_tweet(norm_tweet))
    
    return clean_troll_tweets, clean_norm_tweets


def clean_tweet(tweet, repl={'‘':"'", '’':"'", '“':'"', '”':'"'}):
    #normalize spaces, quotes, double quotes, etc ‘’“”
    for f,r in repl.items():
        tweet = tweet.replace(f,r)
        
    #norm spaces
    tweet = re.sub('  +', ' ', tweet)
        
    #replace links with <LINK>, replace usertags with <USER>, leave hashtags
    tweet = re.sub('https?:[^ ]+', '<LINK>', tweet)
    
    tweet = re.sub('@[a-zA-Z][^ ]+', '<USER>', tweet)
    
    clean = []
    for char in tweet:
        if char in UNICODE_EMOJI['en']:
            clean.append('<EMOJI>')
        else:
            clean.append(char)
            
    return ''.join(clean)

In [42]:
clean_troll_tweets, clean_norm_tweets = clean_tweets(troll_tweets, norm_tweets)

In [43]:
with gzip.open('%s/troll_tweets_clean.pkl.gz' % data_path, 'wb') as oz:
    pickle.dump(clean_troll_tweets, oz)

with gzip.open('%s/normal_tweets_clean.pkl.gz' % data_path, 'wb') as oz:
    pickle.dump(clean_norm_tweets, oz)

In [40]:
for i,ctt in enumerate(clean_norm_tweets):
    if i>100:
        break
    print(ctt)

NormalUser	<USER> coo. thanks. just dropped you a line.
NormalUser	<USER> shit it ain't lettin me DM you back, what's your email?
NormalUser	<USER> hey cody, quick question...can you dm me?
NormalUser	<USER> dang. you need anything? I got some left over meds!
NormalUser	maybe i'm late in the game on this one, but this lowender vst is making my apt rumble!
NormalUser	i really hope A.I. makes the most of this second chance in philly. i'm glad he's goin home.
NormalUser	<USER> danny boy! wanna check out d-nice at the afex 1 year tonight? we could pre-game at mine and walk over.
NormalUser	...and if you have ppl that you care about, make sure to let them know. life is too short to lose friends over bullshit. peace and love.
NormalUser	...that shit weighs heavy on me. take responsibility for your life. I don't blame anyone for where I am in this world...
NormalUser	...including his last failed relationship. and while I know that none of it is grounded in reality, and I actually am worried a

In [15]:
special_tags = set(['<USER>', '<LINK>', '<EMOJI>'])

In [22]:
def tokenize_tweets(troll_tweets, user_tweets,
                    get_lemmas=False, get_pos=False, get_phrases=False, get_ents=False):
    tagged_troll_tweets = []
    for i,troll_tweet in enumerate(troll_tweets):
        tweet_type, tweet_text = troll_tweet.split('\t')
        toks, lemmas, pos, phrases, ents = tokenize_text(tweet_text, get_lemmas, get_pos, get_phrases, get_ents)
        tagged_troll_tweets.append('%s\t%s\t%s\t%s\t%s\t%s\t%s' % (tweet_type, tweet_text, toks, lemmas, pos, phrases, ents))
        
        if i%100000==0:
            print(i, tagged_troll_tweets[-1])
            with gzip.open('%s/troll_tweets_tagged.pkl.gz' % data_path, 'wb') as oz:
                pickle.dump(tagged_troll_tweets, oz)
        
    tagged_user_tweets = []
    for i,user_tweet in enumerate(user_tweets):
        tweet_type, tweet_text = user_tweet.split('\t')
        toks, lemmas, pos, phrases, ents = tokenize_text(tweet_text, get_lemmas, get_pos, get_phrases, get_ents)
        tagged_user_tweets.append('%s\t%s\t%s\t%s\t%s\t%s\t%s' % (tweet_type, tweet_text, toks, lemmas, pos, phrases, ents))
        
        if i%100000==0:
            print(i, tagged_user_tweets[-1])
            with gzip.open('%s/user_tweets_tagged.pkl.gz' % data_path, 'wb') as oz:
                pickle.dump(tagged_user_tweets, oz)
    
    return tagged_troll_tweets, tagged_user_tweets


def tokenize_text(text, get_lemmas=False, get_pos=False, get_phrases=False, get_ents=False):
    toks = []
    lemmas = []
    pos = []
    phrases = []
    ents = []
    
    doc = spacy_nlp(text)
    if get_phrases:
        for chunk in doc.noun_chunks:
            if not ' ' in chunk.text:
                continue
            phrases.append(chunk.text.replace(' ', '_'))
        
    ent_type=''
    ent=[]
    for tok in doc:
        toks.append(tok.text)
        
        if tok.text in special_tags:
            if get_lemmas:
                lemmas.append(tok.text)
            if get_pos:
                pos.append('TAG')
            continue
        
        if get_ents:
            if tok.ent_iob == 3: #start
                ent_type = tok.ent_type_
                ent.append(tok.text)
            elif tok.ent_iob == 1: #continue
                ent.append(tok.text)
            else:
                if ent:
                    ents.append('%s:%s' % ('_'.join(ent), ent_type))
                    ent=[]
                    ent_type=''
                    
        if get_lemmas:
            lemmas.append(tok.lemma_)
            
        if get_pos:
            pos.append(tok.tag_)
        
    toks = ' '.join(toks)
    lemmas = ' '.join(lemmas)
    pos = ' '.join(pos)
    phrases = ' '.join(phrases)
    ents = ' '.join(ents)
    
    return toks, lemmas, pos, phrases, ents

In [7]:
with gzip.open('%s/troll_tweets_clean.pkl.gz' % data_path, 'rb') as fz:
    clean_troll_tweets = pickle.load(fz)

with gzip.open('%s/user_tweets_clean.pkl.gz' % data_path, 'rb') as fz:
    clean_user_tweets = pickle.load(fz)

In [23]:
spacy_nlp.tokenizer.add_special_case(f"<USER>", [{spacy.attrs.ORTH: f"<USER>"}])
spacy_nlp.tokenizer.add_special_case(f"<LINK>", [{spacy.attrs.ORTH: f"<LINK>"}])
spacy_nlp.tokenizer.add_special_case(f"<EMOJI>", [{spacy.attrs.ORTH: f"<EMOJI>"}])

In [ ]:
get_lemmas=True
get_pos=True
get_phrases=True
get_ents=True

tagged_troll_tweets, tagged_user_tweets = tokenize_tweets(clean_troll_tweets, clean_user_tweets,
                                                          get_lemmas, get_pos, get_phrases, get_ents)

0 RightTroll	"We have a sitting Democrat US Senator on trial for corruption and you've barely heard a peep from the mainstream media." ~ <USER> <LINK>	" We have a sitting Democrat US Senator on trial for corruption and you 've barely heard a peep from the mainstream media . " ~ <USER> <LINK>	" we have a sit Democrat US Senator on trial for corruption and you 've barely hear a peep from the mainstream medium . " ~ <USER> <LINK>	`` PRP VBP DT VBG NNP NNP NNP IN NN IN NN CC PRP VBP RB VBN DT NN IN DT NN NNS . '' NFP TAG TAG	a_sitting_Democrat_US_Senator a_peep the_mainstream_media	Democrat_US:GPE
100000 RightTroll	1%ers defrauded People of their President in Primary. If wise they'll give We The People something if a #12thAmendmentElection doesn't occur <LINK>	1%ers defrauded People of their President in Primary . If wise they 'll give We The People something if a # 12thAmendmentElection does n't occur <LINK>	1%er defraud People of their President in Primary . if wise they 'll give we the 

In [ ]:
with gzip.open('%s/troll_tweets_tagged.pkl.gz' % data_path, 'wb') as oz:
    pickle.dump(tagged_troll_tweets, oz)
    
with gzip.open('%s/user_tweets_tagged.pkl.gz' % data_path, 'wb') as oz:
    pickle.dump(tagged_user_tweets, oz)

In [ ]:
#
# Data Analysis
#

In [15]:
# compare histograms
# - lens of troll vs normal tweets
# - user mentions, link mentions
# - given a dict, percent of words in tweet not in the dict

In [15]:
spacy_nlp = spacy.load('en_core_web_sm')

vocab = list(spacy_nlp.vocab.strings)
len(vocab)

83839

In [16]:
# do ngram frequency analysis

In [20]:
#emojis used:
#❤️💙💜♥️⌚️🎉💥🔥♀️💯🚨⚡️🌲🙀👹🍄🌸🌻🌺🌼🌹❄️⛄️🐝🐟🐀🦌🦅🐘🦄🗽
#🙏👊👉👍👌☝️✌👋👇👏💁🏼🤷🏼🙋🚶
#😎😱😨😞☹😳😭😮😡😢😊😂🤣😅🤔🙄😬😁😖🤕😤😜
#👀⇩🎥🎣☠️💀🎶🚂👂3️⃣⏪✅➡️🔁⬇️
#✔️✖️⭕🇺🇸⚔️💣📜🎯✨☕️⚓️�⏱📖📚✪✦🎙